<h1> Counting Summations </h1>

It is possible to write $5$ as a sum in exactly $6$ different ways:
\begin{align*}
    &4 + 1 \\
    &3 + 2 \\
    &3 + 1 + 1 \\
    &2 + 2 + 1 \\
    &2 + 1 + 1 + 1 \\
    &1 + 1 + 1 + 1 + 1 
\end{align*}

How many different ways can $100$ be written as a sum of at least $2$ positive integers?

---

The partition of integers can be described by the recurrence relation,
\begin{gather}
    p_n = \sum_{k=1}^{n} p_k(n) \\
    p_k(n) = p_k(n-k) + p_{k-1}(n-1) \\
    p(n) = \sum_{k \in \mathcal{z}} (-1)^{k+1} p(n-k(3k-1)/2)
\end{gather}
<!-- https://www.whitman.edu/mathematics/cgt_online/book/section03.03.html -->
<!-- https://en.wikipedia.org/wiki/Partition_function_(number_theory) -->

In [2]:
// use std::ops::{Add, Mul, Div};

// extern crate num;
// use num::PrimInt;
// use num::cast::ToPrimitive;

In [3]:

// fn sum_first_n_natural_numbers<T> (n:T) -> T 
//     where T: Add<Output = T> + Mul<Output = T> + Div<Output = T> + PrimInt
// {
//     let two = num::cast(2).unwrap();
//     n*(n + T::one())/two
// }

In [4]:
// fn number_of_summations<T>(n: T) -> T 
//     where T: Mul<Output = T> + Div<Output = T> + PrimInt + std::fmt::Debug
// {
//     if n == num::cast(2).unwrap() {
//         return T::one()
//     }
    
//     let is_even : bool = n & num::one() == num::zero();
//     let two: T = num::cast(2).unwrap();
//     let half_n = n/two;
    
// //     println!("{} {:?}", is_even, half_n);
    
//     if is_even {
//         two*sum_first_n_natural_numbers(half_n) - half_n
//     } else { 
//         two*sum_first_n_natural_numbers(half_n)  
//     }
    
// }

## Brute force

In [5]:
(1..10).for_each(|x| println!("{} {}",x*(3*x-1)/2, x*(3*x-1)/2+x));

1 2
5 7
12 15
22 26
35 40


In [6]:
fn pure_recursive_partition_function(input_num: i32) -> i32 
{
    // Put lowest cases into match to reduce computational time
    match input_num {
        0 | 1 => return 1,
        2 => return 2,
        3 => return 3,
        4 => return 5,
        5 => return 7,
        6 => return 11,
        num if num < 0 => return 0,
        _ => ()
    };
    
    let mut sum = 0;
    let mut k = 1;
    
    loop {
        let x: i32 = (k*(3*k - 1))/2;
        
        if x > input_num + k {
            return sum;
        }
        
        println!("input num = {} {} {}",input_num, input_num - x, input_num -x -k);
        
        let y = pure_recursive_partition_function(input_num - x) + pure_recursive_partition_function(input_num - x - k);
        
        if (k+1) & 1 == 1 {
            sum -= y
        } else {
            sum += y
        }
        
        k += 1;
    }
}

51 57
70 77
92 100
117 126


In [7]:
let soln = pure_recursive_partition_function(12);
println!("p(40) = {}", soln);

input num = 12 11 10
input num = 11 10 9
input num = 10 9 8
input num = 9 8 7
input num = 8 7 6
input num = 7 6 5
input num = 7 2 0
input num = 8 3 1
input num = 7 6 5
input num = 7 2 0
input num = 9 4 2
input num = 9 -3 -6
input num = 8 7 6
input num = 7 6 5
input num = 7 2 0


## A bit less brute force

In [8]:
// use std::collections::HashSet;

// fn required_integers(input_num: u32) -> HashSet<u32> {
    
    
// //     if input_num == 1 {
// //         out.insert(1);
// //         return out;
// //     } else if input_num == 2{
// //         out.insert(1);
// //         out.insert(2);
// //         return out;
// //     }
    
//     if input_num == 1 || input_num == 0 {
//         return HashSet::new();
//     }
    
//     let mut out;
    
//     out = HashSet::with_capacity(input_num as usize);
    
// //     let out: HashSet<u32> = HashSet::with_capacity(input_num as usize);
//     let max_k = ((-1.0_f32 + (1.0 + 24_f32*input_num as f32).sqrt())/6.0) as usize;
//     println!("input num = {} max k = {}", input_num, max_k);
    
//     if max_k < 2 {
//         out.insert(input_num - 1);
//         out.insert(input_num - 2);
//         return out;
//     }
    
//     for k in 1..=max_k {
//         let higher_val = ((3*k*k + k)/2) as u32;
//         let lower_val = ((3*k*k - k)/2) as u32;
        
//         if higher_val <= input_num {
//             out.insert(input_num - higher_val);
//             out.insert(input_num - lower_val);
//         } else if lower_val <= input_num {
//             out.insert(input_num - lower_val);
//         }
//         println!("k = {} set = {:?}",k, out);
        
//     };
    
    
//     let mut updated_vals: HashSet<u32> = out.clone();
    
//     for item in out.iter() {
//         if item > &8 {
// //             let child_vals: HashSet<u32> = required_integers(*item).union(&out).into_iter().map(|x| x.clone()).collect();
//             let child_vals: HashSet<u32> = required_integers(*item);
// //             out = out.union(&&child_vals).collect();
//             updated_vals = out.union(&child_vals).into_iter().map(|x| x.clone()).collect::<HashSet<u32>>();
//         }
// //         let child_vals = required_integers(*item);
// //         out.union(&child_vals);
//     }
    
    
// //         out.union(&required_integers(higher_val));
// //         out.union(&required_integers(lower_val));
//     updated_vals.shrink_to_fit();
    
//     updated_vals
// }

input num = 8 3 1
input num = 10 5 3
input num = 10 -2 -5
input num = 9 8 7
input num = 8 7 6
input num = 7 6 5
input num = 7 2 0
input num = 8 3 1


In [9]:
fn higher_pentagonal_num(num: u32) -> u32 {
    return ((3*num*num + num)/2) as u32
}
fn lower_pentagonal_num(num: u32) -> u32 {
    return ((3*num*num - num)/2) as u32
}


input num = 7 6 5
input num = 7 2 0
input num = 9 4 2
input num = 9 -3 -6
input num = 11 6 4
input num = 11 -1 -4
input num = 10 9 8
input num = 9 8 7
input num = 8 7 6
input num = 7 6 5
input num = 7 2 0
input num = 8 3 1
input num = 7 6 5
input num = 7 2 0
input num = 9 4 2
input num = 9 -3 -6
input num = 8 7 6
input num = 7 6 5
input num = 7 2 0
input num = 8 3 1
input num = 10 5 3
input num = 10 -2 -5
input num = 12 7 5
input num = 7 6 5
input num = 7 2 0
input num = 12 0 -3
p(40) = 77


In [10]:
fn maximum_k_val(input_num: u32) -> u32 {
//     ((-1.0_f32 + (1.0 + 24_f32*input_num as f32).sqrt())/6.0) as u32
//     ((3.0_f32 + (9.0 + 24_f32*input_num as f32).sqrt())/6.0) as u32
    ((1.0_f32 + (1.0 + 24_f32*input_num as f32).sqrt())/6.0) as u32
}

In [11]:
fn compute_pentagonal_num(input_num: u32) -> Vec<u32> {
    let max_k = maximum_k_val(input_num);
    
    if max_k == 0 {
        return Vec::<u32>::new();
    }
    
    (1..=max_k).map(|k| higher_pentagonal_num(k))
                .into_iter()
                .chain(
                    (1..=max_k).map(|k| lower_pentagonal_num(k))
                    .into_iter())
                .collect()
}

In [12]:
use std::collections::HashSet;

fn compute_integers_from_pent_num(input_num: u32, pentagonal_nums: Vec<u32>) -> HashSet<u32> {
    
    let mut set = HashSet::with_capacity(input_num as usize);
    
    for pent_num in pentagonal_nums.iter() {
        let mut inner_counter = 1;
        
        loop {
            let n_time_k = inner_counter*pent_num;
            
            if n_time_k >= input_num {
                break
            } else {
                set.insert(input_num - n_time_k);
                inner_counter += 1;
            }
        }
    }
    
    set.shrink_to_fit();
    set
}

In [13]:
use std::collections::HashSet;

fn required_integers(input_num: u32) -> Vec<u32> {
    
//     let max_k = maximum_k_val(input_num);
    
    let pentagonal_nums = compute_pentagonal_num(input_num);
    println!("{:?}", pentagonal_nums);
    
    let set = compute_integers_from_pent_num(input_num, pentagonal_nums);
    
    let mut out = set.into_iter().collect::<Vec<_>>();
    out.sort();
    out
}

In [14]:
// use std::collections::HashMap;

// fn recursive_partition_function(input_num: u32, storage_map: &HashMap<u32, u32>) -> u32 
// {
//     // Put lowest cases into match to reduce computational time
//     match input_num {
//         0 | 1 => return 1,
//         2 => return 2,
//         3 => return 3,
//         4 => return 5,
//         5 => return 7,
//         6 => return 11,
//         _ => ()
//     };
    
//     let max_k = maximum_k_val(input_num);
//     let mut sum: u32 = 0;
    
//     for k in 1..=max_k {
//         let upper_val: u32 = input_num - lower_pentagonal_num(k);
//         let lower_val: u32 = input_num - higher_pentagonal_num(k);
        
//         let mut upper_partition_val: Option<&u32> = storage_map.get(&upper_val);
//         let mut lower_partition_val: Option<&u32> = storage_map.get(&lower_val);
        
//         if upper_partition_val.is_none() {
//             let temp = recursive_partition_function(upper_val, &storage_map);
//             upper_partition_val = Option::Some(&temp);
//         }
        
//         if lower_partition_val.is_none() {
//             let temp = recursive_partition_function(lower_val, &storage_map);
//             lower_partition_val = Option::Some(&temp);
//         }
        
// //         let y = (upper_partition_val.unwrap() + lower_partition_val.unwrap()) as u32;
        
// //         if (k+1) & 1 == 1 {
// //             sum -= y
// //         } else {
// //             sum += y
// //         }
//     }
    
//     sum
    
// }

In [15]:
use std::collections::HashMap;
use std::convert::TryInto;

fn recursive_partition_function(input_num: u32, storage_map: &HashMap<u32, u32>) -> HashMap<u32, u32>
{
    let mut out = HashMap::with_capacity(1 + storage_map.len());
    
    if input_num < 7 {
        // Put lowest cases into match to reduce computational time
        let out_val = match input_num {0 | 1 => 1,
                                        2 => 2,
                                        3 => 3,
                                        4 => 5,
                                        5 => 7,
                                        6 => 11,
                                        _ => 0
        };
        out.insert(input_num, out_val);
        return out;
    }
    
    let max_k = maximum_k_val(input_num);
    let mut sum: i32 = 0;
    
    for k in 1..=max_k {
        let upper_val: i32 = input_num as i32 - lower_pentagonal_num(k) as i32;
        let lower_val: i32 = input_num as i32 - higher_pentagonal_num(k) as i32;
        
        let vals = [upper_val, lower_val];
//         println!("max k = {}, vals={:?}", max_k, vals);
        
        let mut partition_vals = vec![0; 2];
        
        for (counter, value) in vals.iter().enumerate() {
            
            if value == &0 {
                partition_vals[counter] = 1;
                continue
            } else if value < &0 {
                partition_vals[counter] = 0;
                continue
            }
            
            let val: u32 = *value as u32;
            
            let map_val = storage_map.get(&val);
            
            let num_partitions: u32;
            
            if map_val.is_none() && out.get(&val).is_none() {
                let sub_partition_val = pure_recursive_partition_function(*value as i32) as u32;
                num_partitions = sub_partition_val;
                out.insert(val, sub_partition_val);
            } else if out.get(&val).is_some() {
                num_partitions = *out.get(&val).unwrap();
            } else {
                num_partitions = *map_val.unwrap();
            }
            
            partition_vals[counter] = num_partitions;
        }
        
        
        let temp = (partition_vals[0] + partition_vals[1]) as i32;
        
//         println!("num={}, k={}, pval={:?}, psum={}", input_num, k, partition_vals, temp);
        
        if (k+1) & 1 == 1 {
            sum -= temp
        } else {
            sum += temp
        }
    }
    
    out.insert(input_num, sum as u32);
    
    out.extend(storage_map);
    
    out
}

In [16]:
fn not_in_storage(input_num: u32, storage_map: HashMap<u32, u32>) -> Vec<u32> {
    let pentagonal_nums = compute_pentagonal_num(input_num);
//     let base_integers = pentagonal_nums.iter().map(|pent_num| input_num - pent_num).collect();
    pentagonal_nums.iter()
                    .map(|pent_num| input_num - pent_num)
                    .filter(|base_int| !storage_map.contains_key(base_int))
                    .collect::<Vec<u32>>()
}

In [17]:
fn build_base_partition_vals(partition_integers: Vec<u32>) -> HashMap<u32, u32> {
    
    let mut all_partition_vals: HashMap<u32, u32> = HashMap::new();
    
    for val in partition_integers.iter() {
        all_partition_vals = recursive_partition_function(*val, &all_partition_vals);
    }
    
    all_partition_vals
}

In [18]:
fn find_integer_partition(input_num: u32) -> u32 {
    
//     let recursive_integers = required_integers(input_num);
    let recursive_integers = (0..input_num).collect::<Vec<u32>>();
    
    let store_partition_vals: HashMap<u32, u32> = build_base_partition_vals(recursive_integers);
    
    let all_partition_vals = recursive_partition_function(input_num, &store_partition_vals);
    
    *all_partition_vals.get(&input_num).unwrap()
}

In [19]:
let a = required_integers(100);
println!("{:?}", a);
// let mut combined = a.clone();
// a.iter().for_each(|val| combined.union(&required_integers(*val)));
// println!("{:?}", a);

// for k in 1..100{
//     println!("{} {}", (3*k*k - k)/2, (3*k*k + k)/2)
// }

[2, 7, 15, 26, 40, 57, 77, 100, 1, 5, 12, 22, 35, 51, 70, 92]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [25]:
let b = find_integer_partition(100);
println!("{:?}", b);

190569292
